In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 17.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import os
import shutil
import random
import yaml
import torch.nn as nn
import csv


In [3]:
random.seed(125)

In [4]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Spliting the dataset into train and validation with labels

In [5]:

source_dataset_path = "/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/train/images"
source_labels_path = "/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/train/labels"
test_data_path = "/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/test/images"

In [6]:
# images file 
images = os.listdir(source_dataset_path)
random.shuffle(images)

In [7]:
# Create the paths for train images and validation images
train_images_path = "dataset/train/images"
train_labels_path = "dataset/train/labels"
val_images_path = "dataset/val/images"
val_labels_path = "dataset/val/labels"

In [8]:
# create the directory if don't exist
os.makedirs(train_images_path, exist_ok = True)
os.makedirs(train_labels_path, exist_ok = True)
os.makedirs(val_images_path, exist_ok = True)
os.makedirs(val_labels_path, exist_ok = True)

In [9]:
# training size = 0.85
train_size = 0.9
split_index = int(len(images)*train_size)
train_images = images[:split_index]
val_images =images[split_index:]

In [10]:
# Copying train images and labels
for image in train_images:
    #paths of destination and source
    src_img_path = os.path.join(source_dataset_path, image)
    dest_img_path = os.path.join(train_images_path, image)

    # Check if image already exists in destination
    if not os.path.exists(dest_img_path):
        # Copy image to new directory
        shutil.copy(src_img_path, dest_img_path)
        # print(f"image copied to the destination file for training: {image}")
    
    # Copy corresponding label
    label_name = os.path.splitext(image)[0] + '.txt' #name of label file
    src_lab_path = os.path.join(source_labels_path, label_name)
    dest_lab_path = os.path.join(train_labels_path, label_name)
    
    if (os.path.exists(src_lab_path) and not os.path.exists(dest_lab_path)):
        shutil.copy(src_lab_path, dest_lab_path)
        # print(f"lable copied to the destination file for training {label_name}")

In [11]:
# Copying train images and labels
for image in val_images:
    #paths of destination and source
    src_img_path = os.path.join(source_dataset_path, image)
    dest_img_path = os.path.join(val_images_path, image)

    # Check if image already exists in destination
    if not os.path.exists(dest_img_path):
        # Copy image to new directory
        shutil.copy(src_img_path, dest_img_path)
        # print(f"image copied to the destination file for validation: {image}")
    
    # Copy corresponding label
    label_name = os.path.splitext(image)[0] + '.txt' #name of label file

    src_lab_path = os.path.join(source_labels_path, label_name) 
    dest_lab_path = os.path.join(val_labels_path, label_name)
    
    if (os.path.exists(src_lab_path) and not os.path.exists(dest_lab_path)):
        shutil.copy(src_lab_path, dest_lab_path)
        # print(f"label copied to the destination file for validation: {label_name}")

In [12]:
# writing a data.yaml
data_yaml = """
train: /kaggle/working/dataset/train/images
val: /kaggle/working/dataset/val/images

nc: 6
names: ["aegypti", "albopictus", "anopheles", "culex", "culiseta", "japonicus/koreicus"]
"""

# Save dataset.yaml
with open("dataset/data.yaml", "w") as f:
    f.write(data_yaml)


In [13]:
model = YOLO("yolov5x6u.pt")
model.info()

100%|██████████| 297M/297M [00:00<00:00, 350MB/s] 


YOLOv5x6u summary: 638 layers, 155,544,432 parameters, 0 gradients, 251.7 GFLOPs


(638, 155544432, 0, 251.654336)

In [ ]:
training_results = model.train(
    data = '/kaggle/working/dataset/data.yaml',
    epochs = 15,
    batch = 16,
    imgsz = 640, #default
    device = [0,1],
    workers = 4,
    project = "runs/output", #path where I want to save the model output
    name = 'mosquito_detection',
    optimizer = 'SGD',
    momentum = 0.935,
    seed = 111,
    cos_lr = True,
    close_mosaic = 0,
    lr0 = 0.001,
    lrf = 0.2,
    # dropout = 0.2,
    plots = True,
    pretrained = False,
    weight_decay = 0.0004,
    #augmentation for the training 
    hsv_h = 0.015,
    hsv_s = 0.5,
    hsv_v = 0.3,
    degrees = 0.4,
    translate = 0.1,
    scale = 0.5,
    # bgr=0.05,
    shear = 0.1,
    mosaic = 1,
    erasing = 0, 
    mixup = 0.2,
    # dropout= 0.2
)

Ultralytics 8.3.44 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov5x6u.pt, data=/kaggle/working/dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=4, project=runs/output, name=mosquito_detection, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=111, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

100%|██████████| 755k/755k [00:00<00:00, 17.7MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      8800  ultralytics.nn.modules.conv.Conv             [3, 80, 6, 2, 2]              
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  4    309120  ultralytics.nn.modules.block.C3              [160, 160, 4]                 
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  8   2259200  ultralytics.nn.modules.block.C3              [320, 320, 8]                 
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1 12  13125120  ultralytics.nn.modules.block.C3              [640, 640, 12]                
  7                  -1  1   5531520  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 70.4MB/s]


AMP: checks passed ✅


...

       4/15      10.4G      1.078      1.055      1.323         13        640: 100%|██████████| 422/422 [05:46<00:00,  1.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:23<00:00,  2.04it/s]


                   all        750        750      0.758      0.482      0.472      0.358

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/15      10.4G      1.068     0.9928      1.311         22        640: 100%|██████████| 422/422 [05:46<00:00,  1.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:22<00:00,  2.08it/s]


                   all        750        750      0.745      0.445      0.468      0.365

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/15      10.4G      1.058     0.9333      1.323         17        640:  28%|██▊       | 120/422 [01:38<04:08,  1.22it/s]

In [ ]:
validation_result = model.val()

In [ ]:
validation_result.box.maps

In [ ]:
prediction = model.predict(source=test_data_path,
                           conf=0.4, 
                           save=True,
                           batch = 16, 
                           imgsz = 640,
                           device = 'cuda:0',
                           iou = 0.6,
                           save_txt = True,
                           save_conf = True,                           
                          )

In [ ]:
def get_missing_labels(test_path, label_path):
    """
    Identifies images in the test path without corresponding labels in the label path.
    """
    test_images_name = os.listdir(test_path)
    label_images_name = os.listdir(label_path)

    test_name = {i.strip().split(".")[0] for i in test_images_name}
    lab_name = {i.strip().split(".")[0] for i in label_images_name}

    missing_labels = test_name - lab_name
    return list(missing_labels)

In [ ]:
def parse_prediction_file(file_path, label_mapping):
    """
    Parses a single prediction file to extract the best prediction based on confidence score.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

        if not lines:
            return None
    
        highest_confidence = -1
        best_prediction = None
    
        for line in lines:
            values = line.strip().split()  # Split line into components
            label = label_mapping[int(values[0])]
            xcenter = float(values[1])
            ycenter = float(values[2])
            bbx_width = float(values[3])
            bbx_height = float(values[4])
            conf = float(values[5])
    
            if conf > highest_confidence:
                highest_confidence = conf
                best_prediction = {
                    'LabelName': label,
                    'Conf': conf,
                    'xcenter': xcenter,
                    'ycenter': ycenter,
                    'bbx_width': bbx_width,
                    'bbx_height': bbx_height
                }
        return best_prediction
    


In [ ]:
def generate_predictions(predictions_path, label_mapping):
    """
    Generates predictions from label files in the predictions path.
    """
    predicted_labels = os.listdir(predictions_path)
    results = []

    for label_file in predicted_labels:
        predicted_image_name = label_file.split(".")[0]
        file_path = os.path.join(predictions_path, label_file)
        best_prediction = parse_prediction_file(file_path, label_mapping)

        if best_prediction:
            best_prediction['ImageID'] = predicted_image_name
            results.append(best_prediction)
    return results

In [ ]:
def handle_missing_images(missing_images, label_mapping):
    """
    Handles missing images by assigning random predictions.
    """
    return [
        {
            'ImageID': image_id,
            'LabelName': random.choice(list(label_mapping.values())),
            'Conf': random.uniform(0.5,1),
            'xcenter': random.uniform(0.1,1),
            'ycenter': random.uniform(0.1,1),
            'bbx_width': random.uniform(0.1,1),
            'bbx_height': random.uniform(0.1,1),
        }
        for image_id in missing_images
    ]

In [ ]:
def save_to_csv(output_file, data):
    """
    Saves the data to a CSV file with incremental IDs.
    """
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['id', 'ImageID', 'LabelName', 'Conf', 'xcenter', 'ycenter', 'bbx_width', 'bbx_height']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write the data
        for idx, row in enumerate(data):
            row_with_id = {'id': idx, **row}
            writer.writerow(row_with_id)

    print(f"CSV file saved as {output_file}")

In [ ]:
label_mapping = {
    0: "aegypti", 1: "albopictus", 2: "anopheles",
    3: "culex", 4: "culiseta", 5: "japonicus/koreicus"
}

In [ ]:
predictions_label_path = "/kaggle/working/runs/output/mosquito_detection3/labels"

predicted_labels = os.listdir(predictions_label_path)

In [ ]:
missing_images = get_missing_labels(test_data_path, predictions_label_path)

In [ ]:
print(len(missing_images))

In [ ]:
predictions = generate_predictions(predictions_label_path, label_mapping)

In [ ]:
missing_predictions = handle_missing_images(missing_images, label_mapping)

In [ ]:
output_file = 'submission_SGD_3.csv'

# Combine predictions
all_predictions = predictions + missing_predictions

# Step 4: Save results to CSV by using the function
save_to_csv(output_file, all_predictions)

In [ ]:
model